## Part 1

Importing Data + Creating helper functions

In [76]:
import json

with open('purchase_orders.json', 'r') as file:
    data = json.load(file)
    
with open('penalty_rates.json', 'r') as file:
    penalty_rates_raw = json.load(file)

In [77]:
def sum_units_and_amounts(data):
    """
    takes in a list of dicts
    returns aggregated amounts and units for each customer
    
    
    {target: {
        units: 1000
        amounts: 10000
    }}
    
    """
    
    aggregated_data = {}
    for entry in data:
        customer = entry['Customer']
        order_number = entry['Order Number']
        units = entry['Units']
        amount = float(entry['Amount'][1:])
        
        if customer in aggregated_data:
            aggregated_data[customer]['units'] += units
            aggregated_data[customer]['amount'] += amount
        else:
            aggregated_data[customer] = {'units': 0, 'amount': 0}
        
    return aggregated_data


def sum_everyone(data):
    aggregated_data = {'units': 0, 'amount': 0}
    for entry in data:
        customer = entry['Customer']
        order_number = entry['Order Number']
        units = entry['Units']
        amount = float(entry['Amount'][1:])
        
        aggregated_data['units'] += units
        aggregated_data['amount'] += amount
        
    return aggregated_data
    
    
def organize_penalties(penalty_rates):
    mapping = {}
    for entry in penalty_rates:
        customer = entry['Customer']
        fill_rate_target = entry['Fill Rate Target']
        penalty = entry['Penalty']


        mapping[customer] = {'fill_rate_target': 0, 'penalty': 0}
        mapping[customer]['fill_rate_target'] = float(fill_rate_target.strip('%'))/100
        mapping[customer]['penalty'] = float(penalty.strip('%')) / 100
    return  mapping


# Part 2
def get_max_penalty(order_data, penalty_rates):

    # aggregated_customer_data[entry]['units'] * penalty_rates[entry]['fill_rate_target'] 
    sum_penalties = 0
    # per unit penalty
    for order in order_data:
        
        units_missing = (order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'])
        penalty_rate = (penalty_rates[order['Customer']]['penalty'])
        per_unit_cost = (float(order['Amount'][1:]))/order['Units']
        
        sum_penalties += units_missing + penalty_rate + per_unit_cost

    return sum_penalties

    

In [78]:
print(sum_units_and_amounts(data))

{'Target': {'units': 2008, 'amount': 2168.0}, 'Walmart': {'units': 1875, 'amount': 2175.0}, 'CVS': {'units': 1890, 'amount': 1918.0}}


In [79]:
print(sum_everyone(data))

{'units': 6076, 'amount': 6673.0}


In [80]:
# maximum penalty if we ship nothing

# amount = (units_missing * penalty_rate * unit_cost)


aggregated_customer_data = sum_units_and_amounts(data)

In [70]:
penalty_rates = organize_penalties(penalty_rates_raw)

penalty_rates

    

{'Target': {'fill_rate_target': 0.9, 'penalty': 0.01},
 'Walmart': {'fill_rate_target': 0.95, 'penalty': 0.02},
 'CVS': {'fill_rate_target': 0.9, 'penalty': 0.01}}

## Part 2

In [117]:
penalties = []

# aggregated_customer_data[entry]['units'] * penalty_rates[entry]['fill_rate_target'] 
sum_penalties = 0
# per unit penalty
for order in data:
    
    units_missing = (order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'])
    penalty_rate = (penalty_rates[order['Customer']]['penalty'])
    per_unit_cost = (float(order['Amount'][1:]))/order['Units']
    
    penalties.append(units_missing * penalty_rate * per_unit_cost)
    
    sum_penalties += units_missing * penalty_rate * per_unit_cost

    

sum_penalties

82.90700000000002

In [56]:
# how to minimize penalty

# fulfill as much as we can (up to fulfillment rate) on the highest unit cost orders
# we have a unit budget of 3459 units

# alternatively, we can start with the highest penalty orders

# ill do both to see which one is better

78.099

## Optimize penalty cost based on highest unit cost orders

In [75]:
# nifty trick i learned from leetcode to sort a dictionary in reverse order based on a key

sort_data_by_cost = sorted(data, key= lambda x: (float(x['Amount'][1:]))/x['Units'], reverse=True)
print(sort_data_by_cost)

[{'Customer': 'Walmart', 'Order Number': 127, 'Units': 104, 'Amount': '$304.00', 'Date': '1/1'}, {'Customer': 'CVS', 'Order Number': 125, 'Units': 102, 'Amount': '$172.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 169, 'Units': 146, 'Amount': '$246.00', 'Date': '1/2'}, {'Customer': 'Target', 'Order Number': 171, 'Units': 148, 'Amount': '$248.00', 'Date': '1/2'}, {'Customer': 'Target', 'Order Number': 129, 'Units': 106, 'Amount': '$156.00', 'Date': '1/1'}, {'Customer': 'Target', 'Order Number': 123, 'Units': 100, 'Amount': '$130.00', 'Date': '1/1'}, {'Customer': 'CVS', 'Order Number': 128, 'Units': 105, 'Amount': '$133.00', 'Date': '1/1'}, {'Customer': 'Target', 'Order Number': 126, 'Units': 103, 'Amount': '$113.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 124, 'Units': 101, 'Amount': '$110.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 130, 'Units': 107, 'Amount': '$107.00', 'Date': '1/1'}, {'Customer': 'CVS', 'Order Number': 131, 'Units': 1

In [91]:
# to minimize the penalty i think a good starting point is to fulfill as many high unit cost orders as we can, and then partially fill others
penalties = 0

units_left = 3459
for order in sort_data_by_cost:
    
    units_fulfilled = min((order['Units'] * penalty_rates[order['Customer']]['fill_rate_target']), units_left)
    units_left -= units_fulfilled
    
    # print(units_fulfilled, order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'] - units_fulfilled)
    
    penalty_rate = (penalty_rates[order['Customer']]['penalty'])
    per_unit_cost = (float(order['Amount'][1:]))/order['Units']
    
    penalties += (order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'] - units_fulfilled) * penalty_rate * per_unit_cost
    print(penalties)
    

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7864999999999942
3.275499999999994
4.463499999999994
5.660499999999994
8.206499999999993
9.421499999999993
10.645499999999993
13.248499999999993
14.490499999999994
15.741499999999993
18.40149999999999
19.67049999999999
20.94849999999999
23.665499999999987
24.961499999999987
26.266499999999986
27.589499999999987


## Optimize Penalty based on Highest Penalty Cost

In [127]:
[order for _, order in sorted(list(zip(penalties, data)))]

TypeError: '<' not supported between instances of 'dict' and 'dict'

In [137]:
X = data
Y = penalties

sort_data_by_penalty = [x for _,x in sorted(zip(Y,X), key=lambda x: x[0], reverse=True)]
print(sort_data_by_penalty)

[{'Customer': 'Walmart', 'Order Number': 127, 'Units': 104, 'Amount': '$304.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 169, 'Units': 146, 'Amount': '$246.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 166, 'Units': 143, 'Amount': '$143.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 163, 'Units': 140, 'Amount': '$140.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 160, 'Units': 137, 'Amount': '$137.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 157, 'Units': 134, 'Amount': '$134.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 154, 'Units': 131, 'Amount': '$131.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 151, 'Units': 128, 'Amount': '$128.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 148, 'Units': 125, 'Amount': '$125.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 145, 'Units': 122, 'Amount': '$122.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number':

In [138]:
print(sort_data_by_cost == sort_data_by_penalty)

False


In [139]:
# to minimize the penalty i think a good starting point is to fulfill as many high unit cost orders as we can, and then partially fill others
penalties = 0

units_left = 3459
for order in sort_data_by_penalty:
    
    units_fulfilled = min((order['Units'] * penalty_rates[order['Customer']]['fill_rate_target']), units_left)
    units_left -= units_fulfilled
    
    # print(units_fulfilled, order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'] - units_fulfilled)
    
    penalty_rate = (penalty_rates[order['Customer']]['penalty'])
    per_unit_cost = (float(order['Amount'][1:]))/order['Units']
    
    penalties += (order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'] - units_fulfilled) * penalty_rate * per_unit_cost
    print(penalties)
    

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.9310000000000029
2.119000000000003
3.2890000000000033
4.459000000000003
5.620000000000003
6.763000000000003
7.897000000000003
9.013000000000003
10.120000000000003
11.209000000000003
12.289000000000003
13.351000000000003
14.404000000000003
15.439000000000004
16.465000000000003
17.482000000000003
18.490000000000002
19.489
20.470000000000002
21.442000000000004
